In [3]:
!pip install boto
!pip install boto3
!pip install python-dotenv

In [1]:

from boto.s3.connection import S3Connection
import ssl

from dotenv import load_dotenv
import os

load_dotenv()

# Fetch AWS credentials from environment variables
access_key = os.getenv('AWS_ACCESS_KEY_ID_1')
secret_key = os.getenv('AWS_SECRET_ACCESS_KEY_1')

# Verify that the credentials are loaded
if not access_key or not secret_key:
    raise ValueError("AWS credentials not found in environment variables")


ssl._create_default_https_context = ssl._create_unverified_context
conn = S3Connection(access_key, secret_key)
bucket = conn.get_bucket('data-lake-turma6-puc')

for key in bucket.list():
  print(key.name.encode('utf-8'))

S3ResponseError: S3ResponseError: 403 Forbidden


In [ ]:
import matplotlib.pyplot as plt
import boto3

def map():
    s3 = boto3.resource('s3', aws_access_key_id= access_key, aws_secret_access_key= secret_key)
    bucket = s3.Bucket('data-lake-turma6-puc')
    cont = 0
    mapa_municipios= {}
    for obj in bucket.objects.filter(Prefix='dados-nao-estruturados/MunicipiosSP.csv'):
        for line in obj.get()['Body'].read().decode('ISO-8859-1').splitlines():
            if(cont == 0):
                cont += 1
                continue;

            dados = line.split(";")

            if(dados[0].strip() == ""):
                break;

            percentual = float(dados[12].replace(",","."))
            mapa_municipios[dados[0].strip() + " " + str(percentual) + " %"] = percentual
            cont += 1

        return mapa_municipios

def reduce_mapa(mapa):
    chave = ""
    valor = 0

    for nome in mapa:
        print(nome + " " + str(mapa[nome]))
        if(mapa[nome] > valor):
            chave = nome
            valor = mapa[nome]
            print(mapa)

    reduce = {}
    reduce[chave] = valor
    return reduce


def grava_imagem(mapa):

    mapaPlot = {}
    cont = 0
    for i in sorted(mapa, key=mapa.get, reverse=True):
        if(cont > 30):
            break;

        cont += 1
        mapaPlot[i] = mapa[i]

    plt.barh(list(mapaPlot.keys()), mapaPlot.values())
    plt.gca().invert_yaxis()
    plt.savefig('escolaridade.png', format='png')
    plt.show()
    plt.close()


mapa = map()
mapa_reduce = reduce_mapa(mapa)
grava_imagem(mapa)

In [ ]:
import boto3
import pandas as pd
from io import StringIO
import re

# Configurar o cliente S3
s3 = boto3.resource('s3', aws_access_key_id= access_key, aws_secret_access_key= secret_key)
bucket = s3.Bucket('data-lake-turma6-puc')

# Nome do arquivo específico que você quer ler
file_key = 'dados-nao-estruturados/formula1.txt'

# Obter o conteúdo do arquivo
obj = bucket.Object(file_key)
content = obj.get()['Body'].read().decode('utf-8')

# Pré-processar o conteúdo para substituir as tabulações por ponto e vírgula
# e garantir que não existam delimitadores duplicados
cleaned_content = re.sub(r'\t+', ';', content)
cleaned_content = re.sub(r';+', ';', cleaned_content)

# Ler o conteúdo como um DataFrame pandas usando o ponto e vírgula como delimitador
df = pd.read_csv(StringIO(cleaned_content), delimiter=';', engine='python', header= None)

# Exibir o DataFrame
print(df.head())

In [ ]:
# Adicionar nomes às colunas
df.columns = ['Year', 'Country', 'Winner', 'Team', 'Unnamed: 4']

# Remover a coluna toda nula
df = df.drop(columns=['Unnamed: 4'])

df.head()

a) Um programa em python que mostre os 5 maiores vencedores da Categoria.

b) Um programa em python que mostre qual pais (nacionalidade) foi mas Vencedora.

c) Um programa em python que mostre qual Piloto venceu mais campeonatos?

d) Qual equipe foi a mais vencedora da categoria.

In [ ]:
df.head(15)

In [ ]:
# Limpar a coluna 'Winner'
df['Winner'] = df['Winner'].apply(lambda x: re.sub(r'^\s*\d+\s*', '', x).strip())
df.head(15)

In [ ]:
def map_generic(df, column):
    return [(row[column], 1) for index, row in df.iterrows()]
# Função genérica para reduce
def reduce_generic(data, top_n):
    counts = {}
    for item in data:
        key = item[0]
        if key in counts:
            counts[key] += 1
        else:
            counts[key] = 1
    sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    return sorted_counts[:top_n]
# Exemplo de uso da função map_generic
def plot_generic(data, title, column):
    df = pd.DataFrame(data, columns=[column, title])
    df[title] = pd.to_numeric(df[title], errors='coerce')
    df.plot(kind='bar', x=column, y=title, figsize=(15, 5))
    plt.title(title)
    # plotar em png
    plt.savefig('/content/' + title + '.png', format='png')
    plt.show()



In [ ]:
# a) 5 maiores vencedores da Categoria
mapped_data_winner = map_generic(df, 'Winner')
result = reduce_generic(mapped_data_winner, 5)
print(result)
plot_generic(result, 'Top 5', 'Winner')

In [ ]:
# b) Um programa em python que mostre qual pais (nacionalidade) foi mas Vencedora.
mapped_data_country = map_generic(df, 'Country')
result = reduce_generic(mapped_data_country, 1)
print(result)

In [ ]:
# c) Um programa em python que mostre qual Piloto venceu mais campeonatos?
mapped_data_team = map_generic(df, 'Winner')
result = reduce_generic(mapped_data_team, 1)
print(result)

In [ ]:
# d) Qual equipe foi a mais vencedora da categoria.
mapped_data_team = map_generic(df, 'Team')
result = reduce_generic(mapped_data_team, 1)
print(result)